In [1]:
using BlackBoxOptim, DelimitedFiles

In [2]:
function load_dataset(path)
    XY = readdlm(path, ';', skipstart=1)
    [(label=XY[rr,end]==1, sample=convert(Vector{Bool}, XY[rr,1:end-1])) for rr in 1:size(XY,1)]
end


function leaf(tree, sample)
    index = 1
    while index <= length(tree)
        feature = tree[index]
        if feature == 0
            break
        else
            index = sample[feature] ? 2*index+1 : 2*index
        end
    end
    index
end


function accuracy(tree, dataset)
    
    N = (length(tree)+1)*2
    positives = zeros(Int, N)
    negatives = zeros(Int, N)

    for (label, sample) in dataset
        if label
            positives[leaf(tree, sample)] += 1
        else
            negatives[leaf(tree, sample)] += 1
        end
    end
    
    correct = 0
    for (pp, nn) in zip(positives, negatives)
        correct += pp > nn ? pp : nn
    end
    
    correct / length(dataset)
    
end


nleaves(tree, index=1) = (index>length(tree)||tree[index]==0) ? 1.0 : nleaves(tree,2*index)+nleaves(tree,2*index+1)


function tree(dataset, λ=0.005, max_tree_depth=5)
    
    nfeatures = length(dataset[1].sample)
    
    fitness(tree) = (1.0-accuracy(tree, dataset), nleaves(tree))
    
    fitness_scalar(fitness) = fitness[1] + λ*fitness[2]
    
    multirun = [
        bboptimize(
            tree->fitness(round.(Int, tree));
            SearchRange = (0.0, Float64(nfeatures)), 
            NumDimensions = 2^max_tree_depth-1,
            MaxTime = 5,
            TraceMode = :silent,
            Method = :borg_moea,
            ϵ = 0.0001, τ = 0.1,
            PopulationSize = 500,
            MaxStepsWithoutProgress = 10^6,
            FitnessScheme=ParetoFitnessScheme{2}(is_minimizing=true, aggregator=fitness_scalar)
        ) for run in 1:24]
    
    best_run = sort(multirun, by=run->fitness_scalar(best_fitness(run)))[1]
    
    return round.(Int, best_candidate(best_run))
end 

tree (generic function with 3 methods)

In [3]:
dataset = load_dataset("./compas-binary.csv");

In [4]:
@time best_tree = tree(dataset);

125.518996 seconds (62.30 M allocations: 7.741 GiB, 1.22% gc time)


In [5]:
println(best_tree)

[12, 4, 0, 0, 11, 8, 0, 6, 2, 8, 0, 1, 1, 0, 6, 5, 1, 11, 1, 0, 0, 9, 12, 7, 6, 5, 9, 6, 11, 4, 9]


In [6]:
println("Leaves: ", nleaves(best_tree))
println("Accuracy: ", accuracy(best_tree, dataset))

Leaves: 5.0
Accuracy: 0.669031417402635


In [7]:
# Optimal decision tree generated by OSDT algorithm:
# Hu, Xiyang, Cynthia Rudin, and Margo Seltzer. 
# Optimal sparse decision trees (Figure 4).
# Advances in Neural Information Processing Systems. 2019.

optimal_tree = [12, 4, 0, 0, 11, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
println("Leaves: ", nleaves(optimal_tree))
println("Accuracy: ", accuracy(optimal_tree, dataset))

Leaves: 5.0
Accuracy: 0.669031417402635
